In [43]:
from z3 import *
(x, y) = Ints('x y')
Z = IntSort()
f = Function('f', Z, Z)
s = Solver()
s.add(f(f(x)) == x, f(x) == y, x != y)
print(s.check())
m = s.model()
print("f(f(x)) =", m.evaluate(f(f(x))))
print("f(x)    =", m.evaluate(f(x)))

sat
f(f(x)) = 0
f(x)    = 1


In [18]:
from z3 import *

# DeMorgan
(p, q) = Bools('p q')
demorgan = And(p, q) == Not(Or(Not(p), Not(q)))
print(demorgan)
s = Solver()
s.add(Not(demorgan))
print("Demorgan is...")
if s.check() == unsat:
    print("true :)")
else:
    print("false :(")

And(p, q) == Not(Or(Not(p), Not(q)))
Demorgan is...
true :)


In [23]:
from z3 import *

# Bit Twiddlin'...

def prove(f):
    s = Solver()
    if s.check() == sat:
        print("proved")
    else:
        print("failed to prove")

x = BitVec('x', 32)
powers = [2**i for i in range(32)]
slow   = Or([x == p for p in powers])

print("Proving bit hack...")
fast   = x & (x - 1) == 0
prove(fast == slow)

x = BitVec('x', 32)
y = BitVec('y', 32)

# Claim: (x ^ y) < 0 iff x and y have opposite signs
trick  = (x ^ y) < 0

# Naive way to check if x and y have opposite signs
opposite = Or(And(x < 0, y >= 0), And(x >= 0, y < 0))
print("Proving sign trick...")
prove(trick == opposite)

Proving bit hack...
proved
Proving sign trick...
proved


In [45]:
from z3 import *

# Knapsack: 100 pets for $100
dog, cat, mouse = Ints('dog cat mouse')
solve(dog >= 1,   # at least one dog
      cat >= 1,   # at least one cat
      mouse >= 1, # at least one mouse
      dog + cat + mouse == 100,
      500 * dog + 100 * cat + 25 * mouse == 10000)

[cat = 81, mouse = 16, dog = 3]


In [25]:
from z3 import *

# 8 Queens
# We know each queen must be in a different row.
# So, we represent each queen by a single integer: the column position
Q = [ Int('Q_%i' % (i + 1)) for i in range(8) ]

# Each queen is in a column {1, ... 8 }
val_c = [ And(1 <= Q[i], Q[i] <= 8) for i in range(8) ]

# At most one queen per column
col_c = [ Distinct(Q) ]

# Diagonal constraint
diag_c = [ If(i == j, True,
              And(Q[i] - Q[j] != i - j, Q[i] - Q[j] != j - i))
           for i in range(8) for j in range(i) ]

solve(val_c + col_c + diag_c)

[Q_5 = 1,
 Q_8 = 7,
 Q_3 = 8,
 Q_2 = 2,
 Q_6 = 3,
 Q_4 = 6,
 Q_7 = 5,
 Q_1 = 4]


In [35]:
from z3 import *

# Quantifer Instantiation
(a, b, c, x) = Ints('a b c x')
Z = IntSort()
f = Function('f', Z, Z)
g = Function('g', Z, Z, Z)
print("V x, f(g(x,c)) == a && b==c && g(c,b) --> f(b) == a ... ?")
prove(Implies(And(ForAll(x, f(g(x,c)) == a), b == c, g(c, b) == c), f(b) == a))

V x, f(g(x,c)) == a && b==c && g(c,b) --> f(b) == a ... ?
proved


In [47]:
from z3 import *

# Scopes
(p, q, r) = Bools('p q r')
s = Solver()
# p --> q && !q
s.add(Implies(p, q))
s.add(Not(q))
print(s.check())
s.push()
# p --> q && !q && p
s.add(p)
print(s.check())
s.pop()
print(s.check())

sat
unsat
sat


In [48]:
from z3 import *

# Pareto Optimization
(x, y) = Ints('x y')
opt = Optimize()
opt.set(priority='pareto')
opt.add(x + y == 10, x >= 0, y >= 0)
mx = opt.maximize(x)
my = opt.maximize(y)
while opt.check() == sat:
    print(mx.value(), my.value())

10 0
9 1
8 2
7 3
6 4
5 5
4 6
3 7
2 8
1 9
0 10


In [37]:
from z3 import *

# MaxSat using MaxRes
(a, b, c) = Bools('a b c')
o = Optimize()
# "Hard" constraints (cannot be violated)
o.add(a == c)
o.add(Not(And(a, b)))
# Soft constraints (can be violated, but minimally)
o.add_soft(Or(a, b), 2)
o.add_soft(b, 1)
o.add_soft(c, 1)
print(o.check())
print(o.model())

sat
[c = False, b = True, a = False]


In [50]:
from z3 import *

(i,n, j,N,NNZ) = Ints('i n j N NNZ')
rp = Function('rp', IntSort(), IntSort())
col = Function('col', IntSort(), IntSort())
csr = Solver()
csr.add(N > 0, NNZ > 3, And(0 <= i, i < N))
csr.add(And(0 <= rp(i), rp(i+1) <= NNZ))
csr.add(And(rp(i) <= n, n < rp(i+1)))
csr.add(And(0 <= col(n), col(n) < N))
csr.add(j == col(n))
print(csr)
print(csr.check())
print(csr.model())
#print(csr.sexpr())


[N > 0,
 NNZ > 3,
 And(i >= 0, i < N),
 And(rp(i) >= 0, rp(i + 1) <= NNZ),
 And(rp(i) <= n, n < rp(i + 1)),
 And(col(n) >= 0, col(n) < N),
 j == col(n)]
sat
[i = 0,
 N = 1,
 NNZ = 4,
 n = 3,
 j = 0,
 col = [else -> 0],
 rp = [1 -> 4, else -> 0]]


In [42]:
from z3 import *
import itertools

jobs=[]
"""
# from https://developers.google.com/optimization/scheduling/job_shop
jobs.append([(0, 3), (1, 2), (2, 2)])
jobs.append([(0, 2), (2, 1), (1, 4)])
jobs.append([(1, 4), (2, 3)])

machines=3
makespan=11
"""

#"""
# from http://support.sas.com/documentation/cdl/en/orcpug/63973/HTML/default/viewer.htm#orcpug_clp_sect048.htm
jobs.append([(2,  44),  (3,   5),  (5,  58),  (4,  97),  (0,   9),  (7,  84),  (8,  77),  (9,  96),  (1,  58),  (6,  89)])
jobs.append([(4,  15),  (7,  31),  (1,  87),  (8,  57),  (0,  77),  (3,  85),  (2,  81),  (5,  39),  (9,  73),  (6,  21)])
jobs.append([(9,  82),  (6,  22),  (4,  10),  (3,  70),  (1,  49),  (0,  40),  (8,  34),  (2,  48),  (7,  80),  (5,  71)])
jobs.append([(1,  91),  (2,  17),  (7,  62),  (5,  75),  (8,  47),  (4,  11),  (3,   7),  (6,  72),  (9,  35),  (0,  55)])
jobs.append([(6,  71),  (1,  90),  (3,  75),  (0,  64),  (2,  94),  (8,  15),  (4,  12),  (7,  67),  (9,  20),  (5,  50)])
jobs.append([(7,  70),  (5,  93),  (8,  77),  (2,  29),  (4,  58),  (6,  93),  (3,  68),  (1,  57),  (9,   7),  (0,  52)])
jobs.append([(6,  87),  (1,  63),  (4,  26),  (5,   6),  (2,  82),  (3,  27),  (7,  56),  (8,  48),  (9,  36),  (0,  95)])
jobs.append([(0,  36),  (5,  15),  (8,  41),  (9,  78),  (3,  76),  (6,  84),  (4,  30),  (7,  76),  (2,  36),  (1,   8)])
jobs.append([(5,  88),  (2,  81),  (3,  13),  (6,  82),  (4,  54),  (7,  13),  (8,  29),  (9,  40),  (1,  78),  (0,  75)])
jobs.append([(9,  88),  (4,  54),  (6,  64),  (7,  32),  (0,  52),  (2,   6),  (8,  54),  (5,  82),  (3,   6),  (1,  26)])

machines=10
makespan=842
#"""

# two intervals must not overlap with each other:
def must_not_overlap (s, i1, i2):
    (i1_begin, i1_end)=i1
    (i2_begin, i2_end)=i2
    s.add(Or(i2_begin>=i1_end, i2_begin<i1_begin))
    s.add(Or(i2_end>i1_end, i2_end<=i1_begin))
    (i1_begin, i1_end)=i2
    (i2_begin, i2_end)=i1
    s.add(Or(i2_begin>=i1_end, i2_begin<i1_begin))
    s.add(Or(i2_end>i1_end, i2_end<=i1_begin))

def all_items_in_list_must_not_overlap_each_other(s, lst):
    # enumerate all pairs using Python itertools:
    for pair in itertools.combinations(lst, r=2):
        must_not_overlap(s, (pair[0][1], pair[0][2]), (pair[1][1], pair[1][2]))

s=Solver()

# this is placeholder for tasks, to be indexed by machine number:
tasks_for_machines=[[] for i in range(machines)]

# this is placeholder for jobs, to be indexed by job number:
jobs_array=[]

for job in range(len(jobs)):
    prev_task_end=None
    jobs_array_tmp=[]
    for t in jobs[job]:
        machine=t[0]
        duration=t[1]
        # declare Z3 variables:
        begin=Int('j_%d_task_%d_%d_begin' % (job, machine, duration))
        end=Int('j_%d_task_%d_%d_end' % (job, machine, duration))
        # add variables...
        if (begin,end) not in tasks_for_machines[machine]:
            tasks_for_machines[machine].append((job,begin,end))
        if (begin,end) not in jobs_array_tmp:
            jobs_array_tmp.append((job,begin,end))
        # each task must start at time >= 0
        s.add(begin>=0)
        # end time is fixed with begin time:
        s.add(end==begin+duration)
        # no task must end after makespan:
        s.add(end<=makespan)
        # no task must begin before the end of the last task:
        if prev_task_end!=None:
            s.add(begin>=prev_task_end)
        prev_task_end=end
    jobs_array.append(jobs_array_tmp)

# all tasks on each machine must not overlap each other:
for tasks_for_machine in tasks_for_machines:
    all_items_in_list_must_not_overlap_each_other(s, tasks_for_machine)

# all tasks in each job must not overlap each other:
for jobs_array_tmp in jobs_array:
    all_items_in_list_must_not_overlap_each_other(s, jobs_array_tmp)

if s.check()==unsat:
    print("unsat")

m=s.model()
text_result=[]

# construct Gantt chart:
for machine in range(machines):
    st=[None for i in range(makespan)]
    for task in tasks_for_machines[machine]:
        job=task[0]
        begin=m[task[1]].as_long()
        end=m[task[2]].as_long()
        # fill text string with this job number:
        for i in range(begin,end):
            st[i]=job
    ss=""
    for i,t in enumerate(st):
        ss=ss+("." if t==None else str(st[i]))
    text_result.append(ss)

# we need this juggling to rotate Gantt chart...
print("machines :")
for m in range(len(text_result)):
    print(m)
print("---------")

for time_unit in range(len(text_result[0])):
    print("t=%3d    :" % (time_unit))
    for m in range(len(text_result)):
        print(text_result[m][time_unit])


machines :
0
1
2
3
4
5
6
7
8
9
---------
t=  0    :
7
3
0
.
1
8
6
.
.
9
t=  1    :
7
3
0
.
1
8
6
.
.
9
t=  2    :
7
3
0
.
1
8
6
.
.
9
t=  3    :
7
3
0
.
1
8
6
.
.
9
t=  4    :
7
3
0
.
1
8
6
.
.
9
t=  5    :
7
3
0
.
1
8
6
.
.
9
t=  6    :
7
3
0
.
1
8
6
.
.
9
t=  7    :
7
3
0
.
1
8
6
.
.
9
t=  8    :
7
3
0
.
1
8
6
.
.
9
t=  9    :
7
3
0
.
1
8
6
.
.
9
t= 10    :
7
3
0
.
1
8
6
.
.
9
t= 11    :
7
3
0
.
1
8
6
.
.
9
t= 12    :
7
3
0
.
1
8
6
.
.
9
t= 13    :
7
3
0
.
1
8
6
.
.
9
t= 14    :
7
3
0
.
1
8
6
.
.
9
t= 15    :
7
3
0
.
.
8
6
1
.
9
t= 16    :
7
3
0
.
.
8
6
1
.
9
t= 17    :
7
3
0
.
.
8
6
1
.
9
t= 18    :
7
3
0
.
.
8
6
1
.
9
t= 19    :
7
3
0
.
.
8
6
1
.
9
t= 20    :
7
3
0
.
.
8
6
1
.
9
t= 21    :
7
3
0
.
.
8
6
1
.
9
t= 22    :
7
3
0
.
.
8
6
1
.
9
t= 23    :
7
3
0
.
.
8
6
1
.
9
t= 24    :
7
3
0
.
.
8
6
1
.
9
t= 25    :
7
3
0
.
.
8
6
1
.
9
t= 26    :
7
3
0
.
.
8
6
1
.
9
t= 27    :
7
3
0
.
.
8
6
1
.
9
t= 28    :
7
3
0
.
.
8
6
1
.
9
t= 29    :
7
3
0
.
.
8
6
1
.
9
t= 30    :
7
3
0
.
.
8
6
1
.


5
.
3
.
8
.
t=460    :
.
2
4
6
5
.
3
.
8
.
t=461    :
.
2
4
6
5
.
3
.
8
.
t=462    :
.
2
4
6
5
.
3
.
8
.
t=463    :
.
2
4
6
5
.
3
.
8
.
t=464    :
.
2
4
6
5
.
3
.
8
.
t=465    :
.
2
4
6
5
.
3
.
8
.
t=466    :
.
2
4
6
5
.
3
.
8
.
t=467    :
.
2
4
6
5
.
3
.
8
.
t=468    :
.
2
4
6
5
.
3
.
8
.
t=469    :
.
2
4
6
5
.
3
.
8
.
t=470    :
.
2
4
6
5
.
3
.
8
.
t=471    :
.
2
4
6
5
.
3
.
8
.
t=472    :
.
2
4
6
5
.
3
.
8
.
t=473    :
.
2
4
6
5
.
3
.
8
.
t=474    :
.
2
4
6
5
.
3
.
8
.
t=475    :
.
2
4
6
5
.
3
.
8
.
t=476    :
.
2
4
6
5
.
3
.
8
.
t=477    :
.
2
4
1
5
.
3
.
8
.
t=478    :
.
2
4
1
5
.
3
.
8
.
t=479    :
.
2
4
1
5
.
3
.
8
.
t=480    :
.
2
4
1
5
.
3
.
0
.
t=481    :
.
2
4
1
5
.
3
.
0
.
t=482    :
.
2
4
1
5
.
3
.
0
.
t=483    :
.
2
4
1
5
.
3
.
0
8
t=484    :
.
2
4
1
5
.
3
.
0
8
t=485    :
.
2
4
1
5
.
3
.
0
8
t=486    :
.
2
4
1
5
.
3
.
0
8
t=487    :
.
2
4
1
5
.
3
.
0
8
t=488    :
.
2
4
1
5
.
3
.
0
8
t=489    :
.
2
4
1
5
.
3
.
0
8
t=490    :
.
2
4
1
5
.
3
.
0
8
t=491    :
.
2
4
1
5
.
3
6
